<a href="https://colab.research.google.com/github/vsnupoudel/RandomAssignments/blob/main/LB_Clustering_Read_and_Pickle.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# EDA, Cleaning and Clustering for Forex Clients data

Credentials
- host: technical-test-1.cgajoruzs0oh.ap-southeast-2.rds.amazonaws.com
- port: 5432
- database: technical_test
- user: candidate
- password: NW337AkNQH76veGc

- **Install and restart kernel if needed**

In [ ]:
# !pip install psycopg2
# !pip install yellowbrick

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import psycopg2

In [ ]:
class Connection:
    def __init__(self):
        self.connection = psycopg2.connect(user = "candidate",
                              password = "NW337AkNQH76veGc",
                              host = "technical-test-1.cgajoruzs0oh.ap-southeast-2.rds.amazonaws.com",
                              port = "5432",
                              database = "technical_test")
        
    def queryin(self, query_string):
        with self.connection.cursor()  as cursor: 
            cursor.execute( query_string )
            colnames = [desc[0] for desc in cursor.description]
            result_df = pd.DataFrame(cursor.fetchall(), columns=colnames)
            cursor.close()
        return result_df
    
    def close_conn(self):
        self.connection.close()        

## Tables are small enough to load directly into pd dataframe
- Reading Individual tables as dataframes and merging them in pandas.
- Could be joined directly later,if the table characteristics are known

In [ ]:
connection = Connection()

In [ ]:
select_table_query = '''SELECT column_name, data_type, is_nullable
FROM information_schema.columns
WHERE table_name in ('trades','users') ; '''

connection.queryin(select_table_query)

,column_name,data_type,is_nullable
0,login_hash,text,YES
1,ticket_hash,text,YES
2,server_hash,text,YES
3,symbol,text,YES
4,digits,bigint,YES
5,cmd,bigint,YES
6,volume,bigint,YES
7,open_time,timestamp without time zone,YES
8,open_price,double precision,YES
9,close_time,timestamp without time zone,YES


## From looking at tables
-duplicates in user table

In [ ]:
print(connection.queryin('SELECT count(*) from users ;'))
print(connection.queryin('''select count(*) from (select DISTINCT login_hash,  country_hash, currency, enable 
from users) as sq;'''))

   count
0   1000
   count
0    666


- primary key of trades could be login_hash and ticket_hash

In [ ]:
print( connection.queryin('''select count(*) from (select DISTINCT login_hash, ticket_hash from trades ) as sq;'''))
print( connection.queryin('''select count(*) from trades;''') )

    count
0  100000
    count
0  100000


## Observations
- trades is unique on login_hash and ticket_hash

### Storing users table as dataframe, and running checks on it

In [ ]:
usersdf = connection.queryin('''select DISTINCT login_hash, server_hash, country_hash, currency, enable 
from users;''')

In [ ]:
usersdf[usersdf['currency'] != 'USD'].sample(3)

,login_hash,server_hash,country_hash,currency,enable
513,FAE10F63521309082E08AF7849D3A880,3D1F7E00251C43107EF39F55300781DB,4442E4AF0916F53A07FB8CA9A49B98ED,AUD,1
255,276CB91E08BAACD855A954DFE0E76746,3D1F7E00251C43107EF39F55300781DB,4442E4AF0916F53A07FB8CA9A49B98ED,AUD,1
639,452E3BC11C4F036DF93E93EE4109D930,3D1F7E00251C43107EF39F55300781DB,4442E4AF0916F53A07FB8CA9A49B98ED,AUD,1


In [ ]:
print(usersdf.shape)
print( len(usersdf.login_hash.unique() ) )

(666, 5)
666


- Now the table is unique on login_hash; So, can be joined with the trades table

# Check for null values before joining

In [ ]:
usersdf.isna().sum()

login_hash      0
server_hash     0
country_hash    0
currency        0
enable          0
dtype: int64

In [ ]:
tradesdf = connection.queryin('''SELECT * FROM trades;''')

In [ ]:
print( len(tradesdf.login_hash.unique()) )
print( len(usersdf.login_hash.unique()))

5142
666


In [ ]:
tradesdf.sample(2)

,login_hash,ticket_hash,server_hash,symbol,digits,cmd,volume,open_time,open_price,close_time,contractsize
81427,2369EEC226F9702B25C9EA254443510F,E40898FFA8AAE5238E583C3471522D7D,2F1CA40D76D7E6165DBA3924CE0D3B5C,USDCHF,5,0,10,2020-08-07 16:20:04,0.91550,2020-08-11 12:08:19,100000.0
82815,7060E30AD6D73F360B2D779416EE289C,014D2156BC6A6BA299F5F794FCBF1C93,2F1CA40D76D7E6165DBA3924CE0D3B5C,EURUSD,5,0,2,2020-08-13 17:11:00,1.18625,2020-08-14 09:07:01,100000.0


# Extract data and close connection
- left join on the fact table ( trades)

In [ ]:
output = connection.queryin('''SELECT tr.*, usr.*
                                FROM trades as tr 
                                LEFT JOIN ( select DISTINCT login_hash,  country_hash, currency, enable 
                                from users)  as usr 
                                ON tr.login_hash = usr.login_hash;''')

connection.close_conn()

In [ ]:
opath = 'outputdf.pkl'
output.to_pickle(path= opath )